In [5]:
import findspark

findspark.init()

In [6]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t

In [7]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t

In [8]:
warehouse_location = 'C:/warehouse_location'

spark = (
    SparkSession
    .builder
    .config("spark.driver.memory", '8g')
    .config("spark.sql.warehouse.dir", warehouse_location)
    .enableHiveSupport()
    .getOrCreate()
)

In [9]:
spark.catalog

In [19]:
data_path = 'C:/Users/rodri/Desktop/Dados/data/edc3/'

In [17]:
df_titles = spark.read.csv('title_basics.tsv', header=True, sep='\t')
df_ratings = spark.read.csv('title_ratings.tsv', header=True, sep='\t')


In [20]:
df_titles.write.format('parquet').mode('overwrite').save(data_path + 'title_basics')

In [22]:
df_titles_sample = df_titles.sample(fraction = 0.1)

In [23]:
file_path = 'C:/Users/rodri/Desktop/Dados/data/edc3/title_basics.tsv'

In [ ]:
spark.sql('SHOW DATABASES').toPandas()

In [ ]:
spark.sql('CREATE DATABASE BOOTCAMP;')

In [18]:
spark.sql('USE BOOTCAMP;')

DataFrame[]

In [20]:
spark.sql('SHOW TABLES;').toPandas()

,database,tableName,isTemporary


In [21]:
spark.catalog.listDatabases()

[Database(name='bootcamp', description='', locationUri='file:/C:/warehouse_location/bootcamp.db'),
 Database(name='default', description='Default Hive database', locationUri='file:/C:/warehouse_location')]

In [16]:
spark.sql('USE BOOTCAMP')

DataFrame[]

In [19]:
spark.sql('DROP TABLE title_basics_managed')

DataFrame[]

In [37]:
spark.catalog.listTables()

[]

In [17]:
df_titles_sample.write.saveAsTable("title_basics_managed")

In [20]:
spark.sql("""
CREATE TABLE title_basics_managed 
(tconst STRING, 
 titleType STRING, 
 primaryTitle STRING,
 originalTitle STRING,
 isAdult STRING,
 startYear STRING, 
 endYear STRING,
 runtimeMinutes STRING,
 genres STRING)
""").toPandas()

""


In [23]:
spark.catalog.listTables()

[Table(name='title_basics_managed', database='bootcamp', description=None, tableType='MANAGED', isTemporary=False)]

In [24]:
spark.sql('SHOW TABLES').toPandas()

,database,tableName,isTemporary
0,bootcamp,title_basics_managed,False


In [25]:
spark.sql('SHOW DATABASES').toPandas()

,namespace
0,bootcamp
1,default


In [26]:
spark.sql('USE BOOTCAMP')

DataFrame[]

In [31]:
spark.sql('SELECT * FROM title_basics_managed LIMIT 10').toPandas()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [33]:
spark.sql('USE default')

DataFrame[]

In [34]:
spark.sql('SHOW TABLES')

DataFrame[database: string, tableName: string, isTemporary: boolean]

In [24]:
#configuração#

In [14]:
spark = (
      SparkSession
      .builder
      .config('spark.driver.memory','8g')
      .getOrCreate()
)

In [15]:
spark.conf

In [26]:
df_titles_sample.limit(10).toPandas()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000019,short,The Clown Barber,The Clown Barber,0,1898,\N,\N,"Comedy,Short"
1,tt0000056,short,Uma Corrida de Touros no Campo Pequeno,Uma Corrida de Touros no Campo Pequeno,0,1896,\N,\N,Short
2,tt0000070,short,Demolition of a Wall,Démolition d'un mur,0,1896,\N,1,"Documentary,Short"
3,tt0000077,short,"Le fakir, mystère indien","Le fakir, mystère indien",0,1896,\N,\N,Short
4,tt0000111,short,Rip's Toast,Rip's Toast,0,1896,\N,1,"Drama,Short"
5,tt0000115,short,Réunion d'officiers,Réunion d'officiers,0,1896,\N,\N,"Documentary,Short"
6,tt0000143,short,Les cambrioleurs,Les cambrioleurs,0,1897,\N,1,"Comedy,Short"
7,tt0000165,short,Riña en un café,Riña en un café,0,1897,\N,1,Short
8,tt0000174,short,Výstavní párkar a lepic plakátù,Výstavní párkar a lepic plakátù,0,1898,\N,1,"Comedy,Short"
9,tt0000188,short,The Deserter,The Deserter,0,1898,\N,\N,"Drama,Short"


In [27]:
df_ratings.limit(5).toPandas()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1809
1,tt0000002,6.0,233
2,tt0000003,6.5,1560
3,tt0000004,6.1,152
4,tt0000005,6.2,2383


In [29]:
df_ratings.write.format('parquet').mode('overwrite').save(data_path + 'title_ratings')

In [30]:
int_cols = ['startYear', 'endYear', 'runtimeMinutes', 'isAdult']
for c in int_cols:
    df_titles_sample = (
        df_titles_sample
        .withColumn(c, f.col(c).cast('int'))
    )
# Limpa os Strings
str_cols = ['primaryTitle', 'originalTitle', 'titleType']
for c in str_cols:
    df_titles_sample = (
        df_titles_sample
        .withColumn(c, f.initcap(f.trim(f.col(c))))
    )

In [31]:
df_join = (
    df_titles_sample
    .replace('\\N', None)
    .withColumn('genres', f.split(f.col('genres'), ','))
    .join(df_ratings, 'tconst', 'left')
)

In [33]:
df_join.limit(5).toPandas()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0001581,Short,A Devoted Friend,A Devoted Friend,0,1911,NaN,19.0,[Short],None,None
1,tt0002347,Short,Le Martyre De Saint Étienne,Le Martyre De Saint Étienne,0,1912,NaN,12.0,"[Drama, Short]",None,None
2,tt0002588,Movie,Zigomar Contre Nick Carter,Zigomar Contre Nick Carter,0,1912,NaN,48.0,"[Crime, Thriller]",6.0,32
3,tt0003180,Short,Mutt And Jeff,Mutt And Jeff,0,1913,NaN,NaN,"[Animation, Short]",None,None
4,tt0005793,Movie,Mysteries Of London,Mysteries Of London,0,1915,NaN,NaN,"[Crime, Drama]",6.1,10


In [34]:
df_final = (
    df_join
    .withColumn('genres', f.explode(f.col('genres')))
    .groupBy('titleType')
    .pivot('genres')
    .agg(f.round(f.mean('averageRating'), 2))
    .fillna(0)
)

In [35]:
df_final.limit(5).toPandas()

,titleType,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,Tvepisode,7.50,7.04,7.39,7.31,7.45,7.34,7.55,7.54,7.63,...,6.63,7.02,7.59,7.47,6.98,7.10,6.74,7.51,7.84,7.70
1,Video,6.22,6.52,6.08,6.62,7.10,6.43,6.00,6.97,6.40,...,6.35,6.46,6.26,6.12,6.81,7.24,6.99,5.50,6.13,6.08
2,Videogame,7.00,7.80,7.27,6.85,0.00,7.36,7.19,6.00,7.58,...,0.00,6.40,6.82,7.05,0.00,6.68,0.00,7.12,7.12,6.69
3,Tvminiseries,7.05,6.10,6.87,6.68,7.39,6.90,7.01,7.54,7.06,...,9.00,7.28,6.85,7.07,7.43,7.46,6.39,6.69,7.27,7.15
4,Tvmovie,5.45,6.30,6.05,6.65,6.99,6.52,6.24,7.15,6.58,...,7.97,7.20,6.21,5.77,0.00,6.89,5.60,5.66,7.31,6.34


In [36]:
%%time
df_final.limit(5).toPandas()

Wall time: 48.7 s


,titleType,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,Tvepisode,7.50,7.04,7.39,7.31,7.45,7.34,7.55,7.54,7.63,...,6.63,7.02,7.59,7.47,6.98,7.10,6.74,7.51,7.84,7.70
1,Video,6.22,6.52,6.08,6.62,7.10,6.43,6.00,6.97,6.40,...,6.35,6.46,6.26,6.12,6.81,7.24,6.99,5.50,6.13,6.08
2,Videogame,7.00,7.80,7.27,6.85,0.00,7.36,7.19,6.00,7.58,...,0.00,6.40,6.82,7.05,0.00,6.68,0.00,7.12,7.12,6.69
3,Tvminiseries,7.05,6.10,6.87,6.68,7.39,6.90,7.01,7.54,7.06,...,9.00,7.28,6.85,7.07,7.43,7.46,6.39,6.69,7.27,7.15
4,Tvmovie,5.45,6.30,6.05,6.65,6.99,6.52,6.24,7.15,6.58,...,7.97,7.20,6.21,5.77,0.00,6.89,5.60,5.66,7.31,6.34


In [37]:
%%time
df_join.cache()
df_join.count()

Wall time: 24.3 s


820475

In [38]:
df_final.cache()
df_final.count()

10

In [39]:
%%time
df_final.limit(5).toPandas()

Wall time: 708 ms


,titleType,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,Tvepisode,7.50,7.04,7.39,7.31,7.45,7.34,7.55,7.54,7.63,...,6.63,7.02,7.59,7.47,6.98,7.10,6.74,7.51,7.84,7.70
1,Video,6.22,6.52,6.08,6.62,7.10,6.43,6.00,6.97,6.40,...,6.35,6.46,6.26,6.12,6.81,7.24,6.99,5.50,6.13,6.08
2,Videogame,7.00,7.80,7.27,6.85,0.00,7.36,7.19,6.00,7.58,...,0.00,6.40,6.82,7.05,0.00,6.68,0.00,7.12,7.12,6.69
3,Tvminiseries,7.05,6.10,6.87,6.68,7.39,6.90,7.01,7.54,7.06,...,9.00,7.28,6.85,7.07,7.43,7.46,6.39,6.69,7.27,7.15
4,Tvmovie,5.45,6.30,6.05,6.65,6.99,6.52,6.24,7.15,6.58,...,7.97,7.20,6.21,5.77,0.00,6.89,5.60,5.66,7.31,6.34


In [40]:
df_join.cache()
df_join.count()

820475

In [41]:
%%time
df_join.limit(5).toPandas()

Wall time: 143 ms


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0001581,Short,A Devoted Friend,A Devoted Friend,0,1911,NaN,19.0,[Short],None,None
1,tt0002347,Short,Le Martyre De Saint Étienne,Le Martyre De Saint Étienne,0,1912,NaN,12.0,"[Drama, Short]",None,None
2,tt0002588,Movie,Zigomar Contre Nick Carter,Zigomar Contre Nick Carter,0,1912,NaN,48.0,"[Crime, Thriller]",6.0,32
3,tt0003180,Short,Mutt And Jeff,Mutt And Jeff,0,1913,NaN,NaN,"[Animation, Short]",None,None
4,tt0005793,Movie,Mysteries Of London,Mysteries Of London,0,1915,NaN,NaN,"[Crime, Drama]",6.1,10


In [43]:
#unpersist tira da memoria, persist ou .cache poe na memória#

In [42]:
df_final.unpersist()

DataFrame[titleType: string, Action: double, Adult: double, Adventure: double, Animation: double, Biography: double, Comedy: double, Crime: double, Documentary: double, Drama: double, Family: double, Fantasy: double, Film-Noir: double, Game-Show: double, History: double, Horror: double, Music: double, Musical: double, Mystery: double, News: double, Reality-TV: double, Romance: double, Sci-Fi: double, Short: double, Sport: double, Talk-Show: double, Thriller: double, War: double, Western: double]

In [44]:
spark.catalog.clearCache()

In [45]:
spark